## HTTP Server by Angel Wang

* socket
* bind
* listen
* accept
* recv, sned (may repeat multiple times)
* close socket

In [1]:
import socket
import threading  # multiple threads
import time  

### socket

In [2]:
# use IP (Internet Protocol) as my L3 protocol
# use TCP as my L4 protocol

s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

### bind

In [3]:
# ask the OS to bind the created socket to user-specified parameters: (IP, TCP port)

pars = ('127.0.0.1', 80) # you can change the server port to whatever you want
s.bind(pars)

### listen

In [4]:
# server socket waiting for receiving message
# limits how many clients can be connected to this server

s.listen(5)

### recv, send and close function
Note: it is a function declaration. This function will be called after 'accept'

In [5]:
# a new thread is created for every new accepted client

def serveClient(clientsocket, address):
    
    # we need a loop to continuously receive messages from the client
    while True:
        # then receive at most 1024 bytes message and store these bytes in a variable named 'data'
        data = clientsocket.recv(1024)
        print("from client", data)
        
        # data: GET /get.html HTTP/1.1\r\n
        data_decode = data.decode()
        method_sta = 0
        method_end = data_decode.find(' ',method_sta)
        method = data_decode[method_sta:method_end]
        print("Method: " + method)
        start = data_decode.find('/',0)
        end = data_decode.find('l',start)
        end = end+1
        re_url = data_decode[start:end]
        print("URL: " + re_url)       
        
        # time
        gmt_now = time.strftime("%a, %d %b %Y %I:%M:%S GMT", time.gmtime())
                
        # GET 200 OK
        if method == "GET" and re_url == "/get.html":
            response_data = b'<html><body>good.html<body><html>'
            length = len(response_data)            
            response_status = b'HTTP/1.0 200 OK\r\n'
            response_header= '%s%s%s%s%s%s%s%s%d%s' % ('Date: ',
                                                       gmt_now,
                                                       '\r\n',
                                                       'Server: local\r\n',
                                                       'Connection: close\r\n',
                                                       'Accept-Ranges: bytes\r\n',
                                                       'Content-Type: text/html; charset=UTF-8\r\n',
                                                       'Content-Length: ',
                                                       length,
                                                       '\r\n')
            response_header_b = response_header.encode(encoding="utf-8")
            clientsocket.send(response_status)
            clientsocket.send(response_header_b) 
            clientsocket.send(b'\r\n') # to separate headers from body
            clientsocket.send(response_data)

            print("[200 OK Response]: ")
            print(response_status)
            print(response_header_b)
            print(response_data)
            
            clientsocket.close()
            print("closed")
            break
    
        # GET 301 Moved Permanently      
        elif method == "GET" and re_url == "/redirect.html":
            response_data = b'<html><body>Moved to get.html<body><html>'
            length = len(response_data)  
            response_status = b'HTTP/1.0 301 Moved Permanently\r\n'
            clientsocket.send(response_status) 
            print("[301 Redirect Response]: ")
            print(response_status)
            response_header= '%s%s%s%s%s%s%s%s%d%s' % ('Date: ',
                                                       gmt_now,
                                                       '\r\n',
                                                       'Server: local\r\n',
                                                       'Connection: close\r\n',
                                                       'Location: http://127.0.0.1/get.html\r\n',
                                                       'Content-Type: text/html\r\n',
                                                       'Content-Length: ',
                                                       length,
                                                       '\r\n',
                                                      )
            response_header_b = response_header.encode(encoding="utf-8")
            clientsocket.send(response_header_b) 
            clientsocket.send(b'\r\n') # to separate headers from body
            clientsocket.send(response_data)  
            
#             print("[301 Redirect Response]: ")
#             print(response_status)
            print(response_header_b)
            print(response_data)
        
            clientsocket.close()
            print("closed")
            break
            
        # GET 404 Not Found      
        elif method == "GET" and re_url == "/notfound.html":
            response_data = b'<html><body>404 Not Found<body><html>'
            length = len(response_data)            
            response_status = b'HTTP/1.0 404 Not Found\r\n'
            response_header= '%s%s%s%s%s%s%s%d%s' % ('Date: ',
                                                     gmt_now,
                                                     '\r\n',
                                                     'Server: local\r\n',
                                                     'Connection: close\r\n',
                                                     'Content-Type: text/html; charset=UTF-8\r\n',
                                                     'Content-Length: ',
                                                     length,
                                                     '\r\n',
                                                    )
            response_header_b = response_header.encode(encoding="utf-8")
            clientsocket.send(response_status)
            clientsocket.send(response_header_b) 
            clientsocket.send(b'\r\n') # to separate headers from body
            clientsocket.send(response_data)

            print("[404 Not Found Response]: ")
            print(response_status)
            print(response_header_b)
            print(response_data)
            
            clientsocket.close()
            print("closed")
            break
            
        # HEAD    
        elif method == "HEAD" and re_url == "/head.html": 
            response_data = b'<html><body>head.html<body><html>'
            length = len(response_data) 
            response_status = b'HTTP/1.0 200 OK\r\n'          
            response_header= '%s%s%s%s%s%s%s%d%s' % ('Date: ',
                                                     gmt_now,
                                                     '\r\n',
                                                     'Server: local\r\n',
                                                     'Connection: close\r\n',
                                                     'Content-Type: text/html; charset=UTF-8\r\n',
                                                     'Content-Length: ',
                                                     length,
                                                     '\r\n',
                                                    )
            response_header_b = response_header.encode(encoding="utf-8")
            clientsocket.send(response_status)
            clientsocket.send(response_header_b) 
            clientsocket.send(b'\r\n') # to separate headers from body
#             clientsocket.send(response_data)
            
            print("[HEAD]: ")
            print(response_status)
            print(response_header_b)
#             print(response_data)

            clientsocket.close()
            print("closed")
            break
            
        # POST    
        elif method == "POST" and re_url == "/post.html":  
            id_msg = data_decode.find('id=',0)
            id_beg = data_decode.find('=',id_msg)+1
            id_end = id_beg+9
            print(data_decode[id_beg:id_end])
            
            # test
#             msg_or = 'POST /post.html HTTP/1.1\r\nHost: 127.0.0.1\r\nid=105204027\r\n'
#             msg_or = 'POST /post.html HTTP/1.1\r\nHost: 127.0.0.1\r\n\r\n'
#             msg = msg_or.find('id=',0)
#             m_beg = msg_or.find('=',msg)+1
#             m_end = m_beg+9
#             print(msg_or[m_beg:m_end])
#             if msg == -1:
#                 stu_id = 'You did not post your student id correctly.'
#             else:
#                 stu_id = msg_or[m_beg:m_end]
            
            if id_msg == -1:
                stu_id = 'You did not post your student id correctly.'
            else:
                stu_id = data_decode[id_beg:id_end]
            data = '<html><body>%s<body><html>' % (stu_id)
            response_data = data.encode(encoding="utf-8")            
            length = len(response_data)            
            response_status = b'HTTP/1.0 200 OK\r\n'
            response_header= '%s%s%s%s%s%s%s%d%s' % ('Date: ',
                                                     gmt_now,
                                                     '\r\n',
                                                     'Server: local\r\n',
                                                     'Connection: close\r\n',
                                                     'Content-Type: text/html; charset=UTF-8\r\n',
                                                     'Content-Length: ',
                                                     length,
                                                     '\r\n',
                                                    )
            response_header_b = response_header.encode(encoding="utf-8")
            clientsocket.send(response_status)
            clientsocket.send(response_header_b) 
            clientsocket.send(b'\r\n') # to separate headers from body
            clientsocket.send(response_data)

            print("[POST]: ")
            print(response_status)
            print(response_header_b)
            print(response_data)  
            
            clientsocket.close()
            print("closed")
            break

        else:
            print("false")
            clientsocket.close()
            print("closed")
            break
   
        # if the client sends some termination message to the server, the server close the socket 
#         if data == b'close':
#             clientsocket.close()
#             break

### accept

In [ ]:
# we need a loop to keep accepting new clients (until 5 clients are accepted)

while True:
    # accept a new client and get it's information
    (clientsocket, address) = s.accept()
    
    # create a new thread to serve the new client
    threading.Thread(target = serveClient, args = (clientsocket, address)).start()
    
    print("Accepted")